# Markov chain Monte Carlo: Amortized Approximate Likelihood Ratios

In [ ]:
import hypothesis
import torch
import numpy as np
import matplotlib.pyplot as plt

## Forward model

In [ ]:
from hypothesis.simulation import Simulator

class NormalSimulator(Simulator):
    
    def __init__(self):
        super(NormalSimulator, self).__init__()
        
    def forward(self, inputs):
        return torch.randn(inputs.size(0), 1) + inputs

simulator = NormalSimulator()

## Prior

In [ ]:
from torch.distributions.uniform import Uniform

prior = Uniform(-30, 30)

## Ratio estimator architecture

In [ ]:
from hypothesis.nn import ConditionalMLPRatioEstimator as MLPRatioEstimator

activation = torch.nn.ELU
layers = [64, 64, 64]

## Training the ratio estimator

In [ ]:
from hypothesis.util.data import SimulatorDataset
from hypothesis.nn.conditional_ratio_estimator import ConditionalRatioEstimatorCriterion

inputs_shape = (1,)
outputs_shape = (1,)
ratio_estimator = MLPRatioEstimator(inputs_shape, outputs_shape, layers=layers, activation=activation)
ratio_estimator.train()
dataset = SimulatorDataset(simulator, prior)
batch_size = 128
criterion = ConditionalRatioEstimatorCriterion(ratio_estimator, batch_size)
optimizer = torch.optim.Adam(ratio_estimator.parameters())
epochs = 25

losses = []
for epoch in range(epochs):
    data_loader = torch.utils.data.DataLoader(dataset, num_workers=2, batch_size=batch_size, drop_last=True)
    num_batches = len(data_loader)
    data_loader = iter(data_loader)
    for batch_index in range(num_batches):
        optimizer.zero_grad()
        inputs, outputs = next(data_loader)
        loss = criterion(inputs, outputs)
        loss.backward()
        optimizer.step()
    losses.append(loss.cpu().item())
    
losses = np.array(losses)
plt.plot(np.arange(epochs), np.log(losses), lw=2, color="black")
plt.show()

In some cases the criterion based on the logits might be more numerically preferable.

In [ ]:
from hypothesis.nn.conditional_ratio_estimator import ConditionalRatioEstimatorLogitsCriterion

ratio_estimator = MLPRatioEstimator(inputs_shape, outputs_shape, layers=layers, activation=activation)
ratio_estimator.train()
criterion = ConditionalRatioEstimatorLogitsCriterion(ratio_estimator, batch_size)
optimizer = torch.optim.Adam(ratio_estimator.parameters())

losses = []
for epoch in range(epochs):
    data_loader = torch.utils.data.DataLoader(dataset, num_workers=2, batch_size=batch_size, drop_last=True)
    num_batches = len(data_loader)
    data_loader = iter(data_loader)
    for batch_index in range(num_batches):
        optimizer.zero_grad()
        inputs, outputs = next(data_loader)
        loss = criterion(inputs, outputs)
        loss.backward()
        optimizer.step()
    losses.append(loss.cpu().item())
    
losses = np.array(losses)
plt.plot(np.arange(epochs), np.log(losses), lw=2, color="black")
plt.show()

## Validating the ratio estimator

## Posterior inference using MCMC